## PART I: Ensure New Image Creation Works

In [ ]:
from rime_sdk import Client

## Setup Experiment

Please enter values for the cluster, its API token, and an image you want to create.

In [ ]:
API_TOKEN = ""                        # CREATE & PASTE API_KEY.
CLUSTER_URL = ""  # PASTE DEDICATED DOMAIN OF RIME SERVICE (eg: rime.example.rbst.io).
IMAGE_NAME = "test_managed_image"                       # SELECT A UNIQUE IMAGE NAME
REQ_NAME = "pyjokes"
REQ_VERSION_SPEC="==0.6.0"

In [ ]:
rime_client = Client(CLUSTER_URL, API_TOKEN)

In [ ]:
# Start a new image building job if it doesn't exist
if not rime_client.has_managed_image(IMAGE_NAME, check_status=True):
    # e.g., if the image build job failed
    if rime_client.has_managed_image(IMAGE_NAME):
        rime_client.delete_managed_image(IMAGE_NAME)
    
    job = rime_client.create_managed_image(IMAGE_NAME, [Client.pip_requirement(name=REQ_NAME, version_specifier=REQ_VERSION_SPEC)])

    # Wait until the job has finished and print out status information.
    # Once this prints out the `READY` status, your image is available for use in stress tests.
    job.get_status(verbose=True, wait_until_finish=True)

If that succeeded, make sure your image is created by running:

In [ ]:
print([(i['name'], i['rime_tag']) for i in rime_client.list_managed_images()])

If that succeeded, start a new stress test with your image:

In [ ]:
proj = rime_client.get_project(project_id="11b99073-b1f5-49ed-b6b2-0211afc30c73")

In [ ]:
ds_config = {"connection_info": {"data_file": {"path": "s3://rime-datasets/titanic/titanic_example.csv",},}, "data_params": {"label_col": "Survived",},}
dataset_name = "titanic"
if proj.has_dataset(dataset_name=dataset_name):
    ds_id = proj.get_dataset(dataset_name=dataset_name)['dataset_id']
else:
    ds_id = proj.register_dataset(dataset_name, ds_config)

In [ ]:
model_config = {"model_path": {"path": "s3://rime-models/titanic_s3_test/titanic_example_model.py",},}
model_name = "titanic"
def _has_model(project, model_name):
    return model_name in [x['name'] for x in list(project.list_models())]
if _has_model(proj, model_name):
    mod_id = proj.get_model(model_name=model_name)['model_id']['uuid']
else:
    mod_id = proj.register_model(model_name, model_config)

In [ ]:
config = {"run_name": "Titanic", "data_info": {"ref_dataset_id": ds_id, "eval_dataset_id": ds_id}, "model_id": mod_id, "run_time_info": {"custom_image": {"managed_image_name": IMAGE_NAME}}}

In [ ]:
stress_job = rime_client.start_stress_test(test_run_config=config, project_id=proj.project_id)

In [ ]:
stress_job.get_status(wait_until_finish=True, verbose=True)